In [56]:
from load_data import *
from binary_architecture import *
train_images, train_labels, test_images, test_labels = load_n_images(1200)
# make train_labels and test_labels binary: 0-4 -> 0, 5-9 -> 1
train_labels = np.array([0 if x < 5 else 1 for x in train_labels])
test_labels = np.array([0 if x < 5 else 1 for x in test_labels])

In [42]:
def one_cnn_run(args, seed, train_images, train_labels, test_images, test_labels):
    # Set seeds for reproducibility
    np.random.seed(seed)
    torch.manual_seed(seed)

    # Prepare data loaders
    train_dataset = TensorDataset(torch.Tensor(train_images).unsqueeze(1), torch.Tensor(train_labels).long())
    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)

    test_dataset = TensorDataset(torch.Tensor(test_images).unsqueeze(1), torch.Tensor(test_labels).long())
    test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

    # Instantiate models
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    encoder = Encoder(args).to(device)
    classifier = Class_out(args).to(device)
    conf_out = Conf_out(args).to(device)

    # Define loss functions and optimizer
    criterion_class = nn.CrossEntropyLoss()
    criterion_conf = nn.BCELoss()  # Binary Cross Entropy for confidence prediction
    optimizer = optim.Adam(list(encoder.parameters()) + list(classifier.parameters()) + list(conf_out.parameters()), lr=args.learning_rate)

    # Run the CNN denoise training loop
    best_model, test_z, test_conv_flat, stats = CNN_denoise(
        encoder=encoder,
        classifier=classifier,
        conf_out=conf_out,
        train_loader=train_loader,
        test_loader=test_loader,
        criterion_class=criterion_class,
        criterion_conf=criterion_conf,
        optimizer=optimizer,
        device=device
    )

    # Return or log the results
    results = {
        'best_model': best_model,
        'stats': stats
    }

    one_cnn_stats = stats
    print(one_cnn_stats)
    # Return the results
    return results, test_z, test_conv_flat

In [49]:
best_models = []
loop_num = 10
model_accs = []
model_confs = []

# Hyperparameters
class Args:
    latent_dim = 64 # 10, 32, 64, 128, 256... this would cause PCA testing accuracy to go up
    batch_size = 64
    epochs = 20
    learning_rate = 0.01

args = Args()

# Run the CNN denoise training loop
for seed in range(loop_num):
    results, _, _ = one_cnn_run(args, seed, train_images, train_labels, test_images, test_labels)
    best_models.append(results['best_model'])
    model_accs.append(results['stats']['test_acc'])
    model_confs.append(results['stats']['test_conf'])

{'test_acc': 0.8202, 'test_conf': 0.8176, 'best_loss': 0.5974, 'best_acc': 0.9219, 'best_conf': 0.9301}
{'test_acc': 0.7236, 'test_conf': 0.9195, 'best_loss': 0.4452, 'best_acc': 0.9219, 'best_conf': 0.8804}
{'test_acc': 0.8065, 'test_conf': 0.8856, 'best_loss': 0.7017, 'best_acc': 0.8594, 'best_conf': 0.9198}
{'test_acc': 0.8588, 'test_conf': 0.9697, 'best_loss': 0.2697, 'best_acc': 1.0, 'best_conf': 0.8178}
{'test_acc': 0.842, 'test_conf': 0.9729, 'best_loss': 0.4285, 'best_acc': 0.9219, 'best_conf': 0.9589}
{'test_acc': 0.7577, 'test_conf': 0.8898, 'best_loss': 0.1932, 'best_acc': 1.0, 'best_conf': 0.9354}
{'test_acc': 0.8173, 'test_conf': 0.9338, 'best_loss': 0.495, 'best_acc': 0.9375, 'best_conf': 0.8909}
{'test_acc': 0.8142, 'test_conf': 0.9395, 'best_loss': 0.2688, 'best_acc': 1.0, 'best_conf': 0.8527}
{'test_acc': 0.8588, 'test_conf': 0.9568, 'best_loss': 0.5287, 'best_acc': 0.9062, 'best_conf': 0.9693}
{'test_acc': 0.8616, 'test_conf': 0.9133, 'best_loss': 0.2878, 'best_acc': 

In [53]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def absolute_majority(models, test_loader, args, seed=0):
    np.random.seed(seed)
    torch.manual_seed(seed)
    votes = []
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Initialize and move the model to the device
    encoder = Encoder(args).to(device)
    classifier = Class_out(args).to(device)
    
    
    with torch.no_grad():
        for batch_images, batch_labels in test_loader:
            batch_images = batch_images.to(device)
            batch_votes = []
            
            # Collect votes from each model
            for model in models:
                # Load the weights into the models
                encoder.load_state_dict(model['encoder'])  # Load encoder parameters
                classifier.load_state_dict(model['classifier'])  # Load classifier parameters
                
                encoder.eval()
                classifier.eval()

                z, _ = encoder(batch_images, device)
                class_preds = classifier(z)
                batch_votes.append(torch.argmax(class_preds, dim=1).cpu().numpy())
            
            # Transpose to get votes per sample
            batch_votes = np.array(batch_votes).T  # Shape (batch_size, num_models)
            votes.extend(batch_votes)
    
    # Get the majority vote per sample
    votes = np.array(votes)
    majority_votes = [np.bincount(vote).argmax() for vote in votes]
    
    return majority_votes

# def relative_majority(models, model_accs, test_loader, args, loop_num):
#     votes = []
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     # Initialize and move the model to the device
#     encoder = Encoder(args).to(device)
#     classifier = Class_out(args).to(device)

#     with torch.no_grad():
#         for batch_images, batch_labels in test_loader:
#             batch_images = batch_images.to(device)
#             weighted_votes = np.zeros((batch_images.shape[0], loop_num))
            
#             # Collect weighted votes from each model
#             for i, model in enumerate(models):
#                  # Load the weights into the models
#                 encoder.load_state_dict(model['encoder'])  # Load encoder parameters
#                 classifier.load_state_dict(model['classifier'])  # Load classifier parameters
                
#                 encoder.eval()
#                 classifier.eval()

#                 z, _ = encoder(batch_images, device)
#                 class_preds = classifier(z)
                
#                 # Accumulate weighted probabilities
#                 weighted_votes += model_accs[i] * class_preds.cpu().numpy()
                
#             # Assign the class with the highest weighted vote
#             votes.extend(np.argmax(weighted_votes, axis=1))
    
#     return votes

def evaluate_ensemble(predictions, test_labels):
    correct = (predictions == test_labels).sum()
    total = len(test_labels)
    accuracy = correct / total
    return accuracy

test_dataset = TensorDataset(torch.Tensor(test_images).unsqueeze(1), torch.Tensor(test_labels).long())
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

In [57]:
abs_majority_preds = absolute_majority(best_models, test_loader, args)
print(f'Absolute majority ensemble accuracy: {evaluate_ensemble(abs_majority_preds, test_labels)}')

# rel_majority_preds = relative_majority(best_models, model_accs, test_loader, args, loop_num)
# print(f'Relative majority ensemble accuracy: {evaluate_ensemble(rel_majority_preds, test_labels)}')

Absolute majority ensemble accuracy: 0.8066666666666666


In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

train_pca, test_pca, train_reconstructed, test_reconstructed = PCA_reduction(train_images, test_images, args.latent_dim, random_seed=0)
# Logistic regression classifier for PCA-based latent space
clf_pca = LogisticRegression(max_iter=loop_num)
clf_pca.fit(train_pca, train_labels)  # Train on PCA latent space
test_preds_pca = clf_pca.predict(test_pca)  # Predict on test set
# Compute accuracy
acc_pca = accuracy_score(test_labels, test_preds_pca)
print(f"Accuracy on test set using PCA: {acc_pca * 100:.2f}%")

Accuracy on test set using PCA: 84.00%


### Failed Attempts

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

def vote_majority(X):
    # along the second axis, find the most common element
    return np.array([Counter(column).most_common(1)[0][0] for column in X.T])

# Initialize a list to store the decision trees
decision_trees = []
prediction_trees = []
prediction_trees_probs = []

for i in range(loop_num): 
    # # Create a new decision tree
    tree = DecisionTreeClassifier(max_depth=3, random_state=i, max_features=8) # change max_features according to latent_dim
    # tree = RandomForestClassifier(n_estimators=loop_num, max_depth=5, random_state=i, max_features=64)
    # Fit the decision tree on the latent representations
    training_array = train_zs[i]
    tree.fit(training_array, train_labels)
    # Append the decision tree to the list
    decision_trees.append(tree)

    # Predict the test labels using the decision tree
    testing_array = test_zs[i]
    
    prediction_tree = tree.predict(testing_array)
    prediction_trees.append(prediction_tree)

    # Predict the test labels using the decision tree and calculate the probabilities
    prediction_tree_probs = tree.predict_proba(testing_array)
    prediction_trees_probs.append(prediction_tree_probs)

prediction_trees = np.array(prediction_trees)
prediction_trees_probs = np.array(prediction_trees_probs)

# vote for the majority
rf_predictions = vote_majority(prediction_trees)
# calculate the accuracy
print(f"Testing accuracy: {accuracy_score(test_labels, rf_predictions):.4f}")
# Calculate the average probability for each class across all trees
average_probs = np.mean(prediction_trees_probs, axis=0)
# Calculate the average confidence of the predictions
average_confidences = np.max(average_probs, axis=1)
average_confidence = np.mean(average_confidences)
print(f"Average confidence of predictions: {average_confidence:.4f}")

Testing accuracy: 0.1767
Average confidence of predictions: 0.1321


In [ ]:
# do the same analysis using XGBoost

from xgboost import XGBClassifier

# Initialize a list to store the XGBoost models
xgboost_models = []
prediction_xgboost = []
prediction_xgboost_probs = []

for i in range(loop_num):
    # Create a new XGBoost model
    xgboost = XGBClassifier(n_estimators=loop_num, random_state=i, learning_rate=0.1)
    # Fit the XGBoost model on the latent representations
    training_array = train_zs[i]
    xgboost.fit(training_array, train_labels)
    # Append the XGBoost model to the list
    xgboost_models.append(xgboost)

    # Predict the test labels using the XGBoost model
    testing_array = test_zs[i]
    prediction_xgb = xgboost.predict(testing_array)
    prediction_xgboost.append(prediction_xgb)

    # Predict the test labels using the XGBoost model and calculate the probabilities
    prediction_xgb_probs = xgboost.predict_proba(testing_array)
    prediction_xgboost_probs.append(prediction_xgb_probs)

prediction_xgboost = np.array(prediction_xgboost)
prediction_xgboost_probs = np.array(prediction_xgboost_probs)

# vote for the majority
xgboost_predictions = vote_majority(prediction_xgboost)
# calculate the accuracy
print(f"Testing accuracy: {accuracy_score(test_labels, xgboost_predictions):.4f}")
# Calculate the average probability for each class across all trees
average_probs = np.mean(prediction_xgboost_probs, axis=0)
# Calculate the average confidence of the predictions
average_confidences = np.max(average_probs, axis=1)
average_confidence = np.mean(average_confidences)
print(f"Average confidence of predictions: {average_confidence:.4f}")

Testing accuracy: 0.1333
Average confidence of predictions: 0.1234
